In [1]:
# 1 Vorbereitung: Installiere alle notwendigen Pakete
!pip install -q transformers accelerate datasets peft bitsandbytes evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 10.7 MB/s eta 0:00:00


# Check: Last upadted on 12/06 11:51

In [4]:
# 2 Importieren der Bibliotheken
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
#from trl import SFTTrainer # Aktuell nicht genutzt ##########################################################

In [ ]:
# 3 Hugging Face Connection
from huggingface_hub import login
login()  # hier deinen HF-Token einfügen # HIER ANPASSEN @Adri

In [16]:
# 4 Laden des Datasets und des Tokenizer
from google.colab import drive
drive.mount('/content/drive', force_remount=True) # ggf. ohne mount machen !!! # HIER ANPASSEN @Adri
csv_path = "/content/drive/MyDrive/TrainingMistral/risk_data_formatted_CSV.csv" # HIER ANPASSEN @Adri

# Einlesen der CSV-Datei
df = pd.read_csv(csv_path)
# Test
df.head()

# In HF-Dataset umwandeln
dataset = Dataset.from_pandas(df[['prompt', 'response']])

model_id = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# Festlegen des Padding-Tokens
tokenizer.pad_token = tokenizer.eos_token

Mounted at /content/drive


In [17]:
# 5 Vorbereiten des Datasets

def tokenize(example):
    # 1) Baue die Instruction + Prompt ##########################################################
    instruction = (
        "### Instruction:\n"
        "From the input text, extract the following fields.\n"
        "Format your response as newline-separated entries:\n\n"
        "category: value\n\n"
        "Fields to extract:\n"
        "- risk_communication\n"
        "- single_case_base\n"
        "- absolute_risk_base\n"
        "- absolute_risk_new\n"
        "- absolute_number_base\n"
        "- absolute_number_new\n"
        "- absolute_risk_difference\n"
        "- relative_risk\n"
        "- absolute_number_difference\n"
        "- verbal_descriptor_base\n"
        "- verbal_descriptor_new\n"
        "- verbal_descriptor_change\n"
        "- population_size\n"
        "- reference_class_description_base\n"
        "- reference_class_description_new\n"
        "- reference_class_size_base\n"
        "- reference_class_size_new\n"
        "- source_base\n"
        "- source_new\n"
        "- topic_and_unit\n\n"
    )

    body = f"### Input:\n{example['prompt'].strip()}\n\n### Output:\n"
    full = instruction + body + example['response'].strip()

    # 2) Tokenisiere Prompt-Block (Instruction + Input), um prompt_len zu ermitteln
    tok_prompt = tokenizer(instruction + body, truncation=True, max_length=512) # max_length evtl. höher ansetzen !!!
    prompt_len = len(tok_prompt["input_ids"])

    # 3) Tokenisiere Full-Sequence (Prompt + Response)
    tok_full = tokenizer(full, truncation=True, max_length=512)

    # 4) Baue Labels: -100 für Prompt, echte IDs für Response
    labels = [-100] * prompt_len + tok_full["input_ids"][prompt_len:]

    return {
        "input_ids":      tok_full["input_ids"],
        "attention_mask": tok_full["attention_mask"],
        "labels":         labels
    }

# Anwendung aufs Dataset:
tokenized_ds = dataset.map(tokenize, remove_columns=["prompt","response"], batched=False)


# DataCollator for Seq2Seq -> automatisiert das Padding
from transformers import DataCollatorForSeq2Seq ##########################################################
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    pad_to_multiple_of=8,
    return_tensors="pt"
)

Map:   0%|          | 0/540 [00:00<?, ? examples/s]

Map:   0%|          | 0/540 [00:00<?, ? examples/s]

In [7]:
# 6 BitsAndBytes Config für QLoRA -> double_quant EVTL. ABÄNDERN !!!

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True, # zu False ändern für mehr Genauigkeit? ##########################################################
    bnb_4bit_quant_type="nf4", # richtig? oder standard?
    bnb_4bit_compute_dtype=torch.float16 # ggf. abändern zu torch.bfloat16 -> T4 unterstützt kein bfloat16
)

In [8]:
# 7 Modell laden und vorbereiten

model_id = "mistralai/Mistral-7B-v0.1"  # Beispiel-Checkpoint

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # torch_dtype=torch.bfloat16, # -> im Medium Artikel
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

model.config.use_cache = False # siehe Medium Artikel -> False, um Gradient Checkpointing zu verwenden
model.config.pretraining_tp = 1 # evtl. noch hinzufügen, im Medium Artikel -> Tensor Parallelism -> verteilt auf mehrere GPUs, falls vorhanden
model.gradient_checkpointing_enable() # siehe Medium Artikel
# model.enable_input_require_grads()  # benötigt für QLoRA Training sagt GPT? ##########################################################

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [11]:
# 8 LoRA-Konfiguration
model = prepare_model_for_kbit_training(model) # Hier einfügen -> siehe Medium Artikel !!!


lora_config = LoraConfig(
    r=16, # je höher, desto höhere Anpassungsfähigkeit/ Parameter, aber auch Overfitting Risiko -> Tendenziell RUNTERSCHRAUBEN ##########################################################
    lora_alpha=32, # im richtigen Verhältnis zu r wählen -> Tendenziell RUNTERSCHRAUBEN #################################################################################################
    target_modules=["q_proj", "v_proj"], # k_proj und o_proj hinzufügen? #################################################################################################################
    lora_dropout=0.1, # Vermeidung von Overfitting -> je höher, desto weniger Overfitting, aber potenziell weniger präzises Lernen
    bias="none",
    task_type="CAUSAL_LM"
)


model = get_peft_model(model, lora_config)

In [12]:
# 9 TrainingArguments definieren -> ANPASSEN!!!
training_args = TrainingArguments(
    output_dir="qlora-mistral7b",          # Ordner für Checkpoints
    per_device_train_batch_size=4,         # pro GPU
    gradient_accumulation_steps=8,         # effektive Batch = 4×8 = 32
    learning_rate=1e-4,                    # Start-LR # Empfehlung von Mistral -> ggf. zu 2e-4 ändern
    fp16=True,                             # mixed precision # Falls torch.float16 zu fp16 abändern!!!
    num_train_epochs=3,                    # Epochen
    logging_steps=10,                      # Log alle 10 Schritte
    # eval_strategy="epoch",               # eval nach jeder Epoche ######################################
    # logging_strategy = "steps",
    save_strategy="epoch",                 # speichern nach jeder Epoche
    save_total_limit=2,                    # max. 2 Checkpoints
    report_to="none",                      # verhindert W&B Fehlermeldung
)

In [14]:
# 10 Trainer initialisieren -> EVTL. SFTTrainer VERWENDEN !!!
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds ,
    # eval_dataset = tokenized_val, ##########################################
    # peft_config=lora_config, # Medium Artikel
    # max_seq_length= None, # Medium Artikel
    # dataset_text_field="text", # Medium Artikel
    data_collator=data_collator, # EVTL. RAUSNEHMEN, FALLS PROBLEME MACHT
    tokenizer=tokenizer
    # packing= False, # Medium Artikel
    # compute_metrics = compute_metrics #####################################
)

<ipython-input-14-1993938794>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [15]:
# 11 Training starten
trainer.train()

# wandb API: 01eb4595fbdd612f9d49f834b0d2951f56f7139f

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

Step,Training Loss
10,0.272900
20,0.184300
30,0.159300
40,0.144700
50,0.140100


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=51, training_loss=0.17957338252488306, metrics={'train_runtime': 2720.7159, 'train_samples_per_second': 0.595, 'train_steps_per_second': 0.019, 'total_flos': 3.485126137872384e+16, 'train_loss': 0.17957338252488306, 'epoch': 3.0})

In [ ]:
import matplotlib.pyplot as plt

# Nach trainer.train()
# Trainer-Log-History enthält zwischengespeicherte Loss-Werte
logs = trainer.state.log_history

# Extrahiere nur die Einträge mit 'loss'
steps = [entry['step'] for entry in logs if 'loss' in entry]
losses = [entry['loss'] for entry in logs if 'loss' in entry]

# Plotten
plt.plot(steps, losses)
plt.xlabel('Trainings-Schritt')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.show()


In [20]:
# 12 Adapter und Tokenizer speichern
#model.save_pretrained("mistral7b-qlora-adapter")
trainer.save_model("mistral7b-qlora-adapter")
tokenizer.save_pretrained("mistral7b-qlora-adapter")

# statt in /content/ zu speichern, direkt in dein Drive:
#model.save_pretrained("/content/drive/MyDrive/TrainingMistral/mistral7b-qlora-adapter")
trainer.save_model("/content/drive/MyDrive/TrainingMistral/mistral7b-qlora-adapter")
tokenizer.save_pretrained("/content/drive/MyDrive/TrainingMistral/mistral7b-qlora-adapter")

('/content/drive/MyDrive/TrainingMistral/mistral7b-qlora-adapter/tokenizer_config.json',
 '/content/drive/MyDrive/TrainingMistral/mistral7b-qlora-adapter/special_tokens_map.json',
 '/content/drive/MyDrive/TrainingMistral/mistral7b-qlora-adapter/tokenizer.model',
 '/content/drive/MyDrive/TrainingMistral/mistral7b-qlora-adapter/added_tokens.json',
 '/content/drive/MyDrive/TrainingMistral/mistral7b-qlora-adapter/tokenizer.json')

In [22]:
from transformers import pipeline

# Falls du den Trainer noch in der Session hast:
trained_model = trainer.model
trained_tokenizer = tokenizer

# 🚀 Pipeline anlegen
pipe = pipeline(
    "text-generation",
    model=trained_model,
    tokenizer=trained_tokenizer,
    device_map="auto",        # oder device=0
    torch_dtype=torch.float16 # wenn du mit fp16 trainiert hast
)

# 📝 Deinen Prompt definieren
prompt = (
    "### Instruction:\n"
        "From the input text, extract the following fields.\n"
        "Format your response as newline-separated entries:\n\n"
        "category: value\n\n"
        "Fields to extract:\n"
        "- risk_communication\n"
        "- single_case_base\n"
        "- absolute_risk_base\n"
        "- absolute_risk_new\n"
        "- absolute_number_base\n"
        "- absolute_number_new\n"
        "- absolute_risk_difference\n"
        "- relative_risk\n"
        "- absolute_number_difference\n"
        "- verbal_descriptor_base\n"
        "- verbal_descriptor_new\n"
        "- verbal_descriptor_change\n"
        "- population_size\n"
        "- reference_class_description_base\n"
        "- reference_class_description_new\n"
        "- reference_class_size_base\n"
        "- reference_class_size_new\n"
        "- source_base\n"
        "- source_new\n"
        "- topic_and_unit\n\n"
    "### Input:\n"
    "The risk of heart failure is 10%. Alcohol doubles this risk.\n\n"
    "### Output:\n"
)

# ▶️ Generieren
out = pipe(
    prompt,
    max_new_tokens=400,
    do_sample=False,
    temperature=0
)

print("🔍 Antwort:\n", out[0]["generated_text"])


Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an ex

🔍 Antwort:
 ### Instruction:
From the input text, extract the following fields.
Format your response as newline-separated entries:

category: value

Fields to extract:
- risk_communication
- single_case_base
- absolute_risk_base
- absolute_risk_new
- absolute_number_base
- absolute_number_new
- absolute_risk_difference
- relative_risk
- absolute_number_difference
- verbal_descriptor_base
- verbal_descriptor_new
- verbal_descriptor_change
- population_size
- reference_class_description_base
- reference_class_description_new
- reference_class_size_base
- reference_class_size_new
- source_base
- source_new
- topic_and_unit

### Input:
The risk of heart failure is 10%. Alcohol doubles this risk.

### Output:
risk_communication: 1.0
single_case_base: 0.0
absolute_risk_base: 0.1
absolute_risk_new: null
absolute_number_base: null
absolute_number_new: null
absolute_risk_difference: null
relative_risk: null
absolute_number_difference: null
verbal_descriptor_base: null
verbal_descriptor_new: nul

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=trained_model,
    tokenizer=trained_tokenizer,
    device_map="auto",
    torch_dtype=torch.float16
)

prompt = (
    "### Instruction:\n"
        "From the input text, extract the following fields.\n"
        "Format your response as newline-separated entries:\n\n"
        "category: value\n\n"
        "Fields to extract:\n"
        "- risk_communication\n"
        "- single_case_base\n"
        "- absolute_risk_base\n"
        "- absolute_risk_new\n"
        "- absolute_number_base\n"
        "- absolute_number_new\n"
        "- absolute_risk_difference\n"
        "- relative_risk\n"
        "- absolute_number_difference\n"
        "- verbal_descriptor_base\n"
        "- verbal_descriptor_new\n"
        "- verbal_descriptor_change\n"
        "- population_size\n"
        "- reference_class_description_base\n"
        "- reference_class_description_new\n"
        "- reference_class_size_base\n"
        "- reference_class_size_new\n"
        "- source_base\n"
        "- source_new\n"
        "- topic_and_unit\n\n"
    "### Input:\n"
    "The risk of heart failure is 10%. Alcohol doubles this risk.\n\n"
    "### Output:\n"
)

out = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=False,     # Greedy decoding → deterministisch
    use_cache=False      # passt zu Gradient Checkpointing
)

print("🔍 Antwort:\n", out[0]["generated_text"])


Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an ex

In [ ]:
# 13 Evaluation -> noch nicht geprüft!
model.eval()
test_prompt = "### Input:\nIn a new study, researchers discovered...\n### Output:\n"
inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.7)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))